In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install torch torchvision


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install timm


Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

# 检查 GPU 是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据增强和标准化
transform = transforms.Compose(
    [
        transforms.Resize((32, 32)),  # 调整图像大小为 32x32
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

# 下载 CIFAR-10 数据集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# 简单的 CNN 模型
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # 输入通道数为3，输出通道数为32
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)  # 输入特征数为64 * 8 * 8，输出特征数为256
        self.fc2 = nn.Linear(256, 10)  # 输出10个类别

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # 卷积 + ReLU + 池化
        x = self.pool(torch.relu(self.conv2(x)))  # 卷积 + ReLU + 池化
        x = x.view(-1, 64 * 8 * 8)  # 展平
        x = torch.relu(self.fc1(x))  # 全连接层 + ReLU
        x = self.fc2(x)  # 输出层
        return x

# 初始化模型，定义损失函数和优化器
net = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)  # 使用 Adam 优化器

# 训练模型
def train(num_epochs):
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:  # 每100个小批量输出一次
                print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
                running_loss = 0.0

    print("训练完成")

# 评估模型
def test():
    correct = 0
    total = 0
    with torch.no_grad():  # 在测试时不计算梯度
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'测试准确率: {100 * correct / total:.2f}%')

# 训练模型并评估
start_time = time.time()
train(num_epochs=10)  # 设置训练的 epoch 数
test()
end_time = time.time()

print(f"训练和测试时间: {end_time - start_time:.2f} 秒")


100%|██████████| 170498071/170498071 [00:03<00:00, 48891642.09it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
[1, 100] loss: 1.839
[1, 200] loss: 1.502
[1, 300] loss: 1.390
[1, 400] loss: 1.282
[1, 500] loss: 1.245
[1, 600] loss: 1.193
[1, 700] loss: 1.144
[2, 100] loss: 1.003
[2, 200] loss: 0.980
[2, 300] loss: 0.983
[2, 400] loss: 0.970
[2, 500] loss: 0.938
[2, 600] loss: 0.932
[2, 700] loss: 0.912
[3, 100] loss: 0.818
[3, 200] loss: 0.766
[3, 300] loss: 0.787
[3, 400] loss: 0.797
[3, 500] loss: 0.786
[3, 600] loss: 0.795
[3, 700] loss: 0.777
[4, 100] loss: 0.633
[4, 200] loss: 0.648
[4, 300] loss: 0.664
[4, 400] loss: 0.642
[4, 500] loss: 0.646
[4, 600] loss: 0.669
[4, 700] loss: 0.669
[5, 100] loss: 0.520
[5, 200] loss: 0.501
[5, 300] loss: 0.528
[5, 400] loss: 0.528
[5, 500] loss: 0.544
[5, 600] loss: 0.546
[5, 700] loss: 0.543
[6, 100] loss: 0.396
[6, 200] loss: 0.383
[6, 300] loss: 0.415
[6, 400] loss: 0.396
[6, 500] loss: 0.441
[6, 600] loss: 0.448
[6, 700] loss: 0.445
[7, 100] loss: 0.275
[7, 200]

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time
import timm  # 用于加载模型

# 检查 GPU 是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据增强和标准化
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),  # ResNet 通常使用 224x224 输入大小
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

# 下载 CIFAR-10 数据集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# 使用 ResNet 模型
class ResNetModel(nn.Module):
    def __init__(self):
        super(ResNetModel, self).__init__()
        self.model = timm.create_model('resnet18', pretrained=True)  # 使用预训练的 ResNet-18 模型
        self.model.fc = nn.Linear(self.model.fc.in_features, 10)  # 修改输出层为 10 类别

    def forward(self, x):
        return self.model(x)

# 初始化模型，定义损失函数和优化器
net = ResNetModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)  # 使用 Adam 优化器

# 训练模型
def train(num_epochs):
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:  # 每100个小批量输出一次
                print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
                running_loss = 0.0

    print("训练完成")

# 评估模型
def test():
    correct = 0
    total = 0
    with torch.no_grad():  # 在测试时不计算梯度
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'测试准确率: {100 * correct / total:.2f}%')

# 训练模型并评估
start_time = time.time()
train(num_epochs=10)  # 设置训练的 epoch 数
test()
end_time = time.time()

print(f"训练和测试时间: {end_time - start_time:.2f} 秒")


Files already downloaded and verified
Files already downloaded and verified


model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

[1, 100] loss: 1.027
[1, 200] loss: 0.452
[1, 300] loss: 0.402
[1, 400] loss: 0.311
[1, 500] loss: 0.314
[1, 600] loss: 0.274
[1, 700] loss: 0.270
[2, 100] loss: 0.203
[2, 200] loss: 0.179
[2, 300] loss: 0.201
[2, 400] loss: 0.199
[2, 500] loss: 0.179
[2, 600] loss: 0.170
[2, 700] loss: 0.182
[3, 100] loss: 0.120
[3, 200] loss: 0.118
[3, 300] loss: 0.118
[3, 400] loss: 0.136
[3, 500] loss: 0.132
[3, 600] loss: 0.137
[3, 700] loss: 0.130
[4, 100] loss: 0.093
[4, 200] loss: 0.088
[4, 300] loss: 0.085
[4, 400] loss: 0.100
[4, 500] loss: 0.097
[4, 600] loss: 0.106
[4, 700] loss: 0.102
[5, 100] loss: 0.068
[5, 200] loss: 0.072
[5, 300] loss: 0.077
[5, 400] loss: 0.084
[5, 500] loss: 0.081
[5, 600] loss: 0.087
[5, 700] loss: 0.096
[6, 100] loss: 0.066
[6, 200] loss: 0.065
[6, 300] loss: 0.064
[6, 400] loss: 0.055
[6, 500] loss: 0.056
[6, 600] loss: 0.067
[6, 700] loss: 0.065
[7, 100] loss: 0.046
[7, 200] loss: 0.045
[7, 300] loss: 0.054
[7, 400] loss: 0.057
[7, 500] loss: 0.068
[7, 600] loss

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time
import timm

# 检查 GPU 是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据增强和标准化
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 下载 CIFAR-10 数据集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# 使用 EfficientNet 模型
class EfficientNetModel(nn.Module):
    def __init__(self):
        super(EfficientNetModel, self).__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=True)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, 10)  # 修改输出层为 10 类别

    def forward(self, x):
        return self.model(x)

# 初始化模型，定义损失函数和优化器
net = EfficientNetModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# 训练模型
def train(num_epochs):
    for epoch in range(num_epochs):
        net.train()
        running_loss = 0.0
        for i, data in enumerate(trainloader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:
                print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
                running_loss = 0.0

        scheduler.step()
        save_model(epoch)

    print("训练完成")

# 保存模型
def save_model(epoch):
    torch.save(net.state_dict(), f'efficientnet_b0_epoch_{epoch + 1}.pth')

# 评估模型
def test():
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'测试准确率: {100 * correct / total:.2f}%')

# 训练模型并评估
start_time = time.time()
train(num_epochs=10)
test()
end_time = time.time()

print(f"训练和测试时间: {end_time - start_time:.2f} 秒")


Files already downloaded and verified
Files already downloaded and verified


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

[1, 100] loss: 0.668
[1, 200] loss: 0.371
[1, 300] loss: 0.332
[1, 400] loss: 0.316
[1, 500] loss: 0.270
[1, 600] loss: 0.312
[1, 700] loss: 0.245
[2, 100] loss: 0.181
[2, 200] loss: 0.196
[2, 300] loss: 0.198
[2, 400] loss: 0.179
[2, 500] loss: 0.176
[2, 600] loss: 0.186
[2, 700] loss: 0.179
[3, 100] loss: 0.125
[3, 200] loss: 0.127
[3, 300] loss: 0.137
[3, 400] loss: 0.147
[3, 500] loss: 0.156
[3, 600] loss: 0.146
[3, 700] loss: 0.158
[4, 100] loss: 0.107
[4, 200] loss: 0.108
[4, 300] loss: 0.112
[4, 400] loss: 0.120
[4, 500] loss: 0.124
[4, 600] loss: 0.127
[4, 700] loss: 0.127
[5, 100] loss: 0.094
[5, 200] loss: 0.074
[5, 300] loss: 0.097
[5, 400] loss: 0.093
[5, 500] loss: 0.102
[5, 600] loss: 0.114
[5, 700] loss: 0.105
[6, 100] loss: 0.062
[6, 200] loss: 0.045
[6, 300] loss: 0.040
[6, 400] loss: 0.030
[6, 500] loss: 0.030
[6, 600] loss: 0.028
[6, 700] loss: 0.024
[7, 100] loss: 0.016
[7, 200] loss: 0.015
[7, 300] loss: 0.015
[7, 400] loss: 0.015
[7, 500] loss: 0.015
[7, 600] loss